In [233]:
! pip install qeds
import random
import numpy as np
import pandas as pd
import qeds
import matplotlib.pyplot as plt
%matplotlib inline
import qeds
qeds.themes.mpl_style();

# GroupBy - Exercises

**We will begin with a simple made-up dataset to discuss the concepts and then work through extended example and exercises with real data.**

In [234]:
C = np.arange(1, 7, dtype=float)
C[[3, 5]] = np.nan
df = pd.DataFrame({
    "A" : [1, 1, 1, 2, 2, 2],
    "B" : [1, 1, 2, 2, 1, 1],
    "C": C,
})
df

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


**We create a DataFrameGroupBy to use in what follows.**

In [235]:
gbA = df.groupby("A")
gbA.head()

,A,B,C
0,1,1,1.0
1,1,1,2.0
2,1,2,3.0
3,2,2,NaN
4,2,1,5.0
5,2,1,NaN


## Exercise 1

**Try the following operations: gbA.sum(), gbA.mean(), gbA.count(). How did pandas compute the sum of `gbA`? What happened to the `NaN` entries in column `C`? Write your thoughts.** 

In [236]:
# the NaN entries in column C are disregarded in the analysis, regardless of the operation we use
    # for A = 2: the sum is 5, the mean is also 5 (computed with one element), the count gives us 1 element

print("This is gbA.sum()")
display(gbA.sum())
print("This is gbA.mean()")
display(gbA.mean())
print("This is gbA.count()")
display(gbA.count())

This is gbA.sum()


,B,C
A,,
1,4,6.0
2,4,5.0


This is gbA.mean()


,B,C
A,,
1,1.333333,2.0
2,1.333333,5.0


This is gbA.count()


,B,C
A,,
1,3,3
2,3,1


## Exercise 2

1. **Use introspection (tab completion) to see what other aggregations are defined for GroupBy objects. Pick three and evaluate them in the cells below.**

In [237]:
print("This is gbA.max()")
display(gbA.max())
print("This is gbA.median()")
display(gbA.median())
print("This is gbA.diff()")
display(gbA.diff())

This is gbA.max()


,B,C
A,,
1,2,3.0
2,2,5.0


This is gbA.median()


,B,C
A,,
1,1,2.0
2,1,5.0


This is gbA.diff()


,B,C
0,NaN,NaN
1,0.0,1.0
2,1.0,1.0
3,NaN,NaN
4,-1.0,NaN
5,0.0,NaN


2. **Does the output of each of these commands have the same features as the output of `gbA.sum()` from above? If not, what is different?**

In [238]:
#the three commands chosen seem to have the same features, excet for the last one where the key disappears

## Exercise 3

1. **Write a function that, given a DataFrame, computes each entry's deviation from the mean of its column and apply the function to `gbA`.**

In [239]:
# write function here
def mean_deviation(x):
    avg = x.mean()
   
    return x - avg

# apply function here
mean_deviation = gbA.apply(mean_deviation)
mean_deviation

,B,C
0,-0.333333,-1.0
1,-0.333333,0.0
2,0.666667,1.0
3,0.666667,NaN
4,-0.333333,0.0
5,-0.333333,NaN


2. **Describe what the index and and columns are? Where are the group keys (the `A` column)?**

In [240]:
mean_deviation.info()
#the index contains 6 entries, from 0 to 5
#the columns are B and C
#the group keys no longer appear
    #gbA.groups.keys()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
B    6 non-null float64
C    4 non-null float64
dtypes: float64(2)
memory usage: 224.0 bytes


3. **Determine the correct way to add these results back into `df` as new columns.** 
    - Hint: remember the merge lecture

In [241]:
# add output of function as new columns to df here...
mean_deviation_tu = mean_deviation.rename(columns={"B":"B_dev", "C":"C_dev"},inplace=True)

merged=pd.concat([df, mean_deviation], axis=1)
merged


,A,B,C,B_dev,C_dev
0,1,1,1.0,-0.333333,-1.0
1,1,1,2.0,-0.333333,0.0
2,1,2,3.0,0.666667,1.0
3,2,2,NaN,0.666667,NaN
4,2,1,5.0,-0.333333,0.0
5,2,1,NaN,-0.333333,NaN


<dl style='margin: 20px 0;'>
<dt>Note that if the group keys</dt>
<dd>
remained in the index as the `.apply`'s output, the merge/join step would have been complicated.

## Exercise 4

**We use an airline DataFrame:**

In [242]:
air_dec = qeds.load("airline_performance_dec16")
air_dec.head()

,Carrier,TailNum,FlightNum,OriginCityMarketID,Origin,OriginCityName,OriginStateName,DestCityMarketID,Dest,DestCityName,...,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,Date
0,AA,N3JHAA,46,31650,MSP,"Minneapolis, MN",Minnesota,30977,ORD,"Chicago, IL",...,58.0,334.0,0.0,0.0,20.0,0.0,0.0,NaN,NaN,2016-12-18
1,AA,N3DPAA,46,31650,MSP,"Minneapolis, MN",Minnesota,30977,ORD,"Chicago, IL",...,57.0,334.0,0.0,0.0,20.0,0.0,0.0,NaN,NaN,2016-12-19
2,AA,N3KUAA,46,31650,MSP,"Minneapolis, MN",Minnesota,30977,ORD,"Chicago, IL",...,49.0,334.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2016-12-20
3,AA,N3FBAA,46,31650,MSP,"Minneapolis, MN",Minnesota,30977,ORD,"Chicago, IL",...,51.0,334.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2016-12-21
4,AA,N3BLAA,46,31650,MSP,"Minneapolis, MN",Minnesota,30977,ORD,"Chicago, IL",...,51.0,334.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2016-12-22


1. **Which type of delay was the most common?**

In [243]:
#The air_dec DataFrame has information on the minutes of delay attributed to 5 different categories:
delay_cols = [
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay'
]

air_dec[delay_cols]=(air_dec[delay_cols] > 0) 
air_dec[delay_cols][air_dec[delay_cols]==True].count().sort_values()
#CarrierDelay was the most frequent one

SecurityDelay          356
WeatherDelay          5946
NASDelay             55765
LateAircraftDelay    56377
CarrierDelay         57189
dtype: int64

2. **Which one caused the largest average delay?**  

In [244]:
air_dec[delay_cols].mean().sort_values()
#CarrierDelay caused the largest average delay

SecurityDelay        0.000772
WeatherDelay         0.012899
NASDelay             0.120979
LateAircraftDelay    0.122306
CarrierDelay         0.124068
dtype: float64

3. **Does that vary by airline?**  

In [245]:
air_dec.groupby("Carrier")[delay_cols].mean()
#it does: 
    #AA - LateAircraftDelay
    #EV - CarrierDelay

,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
Carrier,,,,,
AA,0.112612,0.012317,0.118127,0.001233,0.089361
AS,0.067172,0.016121,0.154461,0.002067,0.078333
B6,0.197797,0.005878,0.149540,0.002918,0.158089
DL,0.104186,0.020499,0.099187,0.000168,0.076663
EV,0.113454,0.004558,0.124010,0.000000,0.122304
F9,0.180027,0.010525,0.181006,0.000000,0.154693
HA,0.115488,0.027730,0.002363,0.000945,0.077517
NK,0.117898,0.007484,0.203283,0.001449,0.089731
OO,0.090251,0.016382,0.149895,0.000484,0.150500
